In [ ]:
! pip install transformers accelerate datasets bitsandbytes peft sentencepiece

In [11]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "../data/medquad.csv"})
print(dataset["train"][0])


{'question': 'What is (are) Glaucoma ?', 'answer': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vision. Ope

In [12]:
from datasets import load_dataset

dataset = load_dataset(
    "csv",
    data_files={"train": "/Users/rohansingh/Documents/medicalQNA/data/medquad.csv"}
)

dataset = dataset["train"]
dataset


Dataset({
    features: ['question', 'answer', 'source', 'focus_area'],
    num_rows: 16412
})

## Qwen Preprocessing Format

following chat-style template for preparing instruction–answer pairs for Qwen training:

```text
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
{answer}
<|im_end|>


In [ ]:
def format_chat(example):
    example["text"] = (
        "<|im_start|>user\n"
        f"You are a safe and helpful medical assistant.\n"
        f"Question: {example['question']}\n"
        "<|im_end|>\n"
        "<|im_start|>assistant\n"
        f"{example['answer']}\n"
        "<|im_end|>"
    )
    return example

dataset = dataset.map(format_chat)
dataset = dataset.train_test_split(test_size=0.02)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16
)



In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, lora_config)



In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="./qwen-small-medquad",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    max_steps=800,       # small training
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_steps=200,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

trainer.train()


In [ ]:
model = model.merge_and_unload()
model.save_pretrained("./qwen2.5-1.5B-medquad")
tokenizer.save_pretrained("./qwen2.5-1.5B-medquad")


## Test Inference

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "./qwen2.5-1.5B-medquad"     # saved model

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

def ask_medical(question):
    prompt = (
        "<|im_start|>user\n"
        f"{question}\n"
        "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.2,
        do_sample=False
    )

    text = tokenizer.decode(output[0], skip_special_tokens=True)
    return text.split("assistant")[-1].strip()  # clean response


In [ ]:
print(ask_medical("What are the symptoms of iron deficiency anemia?"))

In [ ]:
print(ask_medical("What causes chronic kidney disease in adults?"))

In [ ]:
print(ask_medical("Explain high blood pressure in simple words."))

In [3]:
! pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.5/861.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 11.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: brotli
    Found existing installation: Brotli 1.0.9
    Uninstalling Brotli-1.0.9:
      Successfully uninstalled Brotli-1.0.9━━━━━━━━━━━━━━━━━━━━━━━  1/19 [brotli]
  Attempting uninstall: typer━━━━━━╺━━━━━━━━━━━━ 13/19 [starlette]
    Found existing installation: typer 0.9.0m━━━━━━━━━━━━ 13/19 [starlette]
    Uninstalling typer-0.9.0:━━╺━━━━━━━━━━━━ 13/19 [starlette]
      Successfully uninstalled typer-0.9.0━━━━━━━━━━━━ 13/19 [starlette]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19/19 [gradio]18/19 [gradio]


In [10]:
! pip install --upgrade bitsandbytes

In [11]:
import gradio as gr
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# ---- CHANGE THIS ----
MODEL_PATH = "/Users/rohansingh/Documents/medicalQNA/project/content/qwen-medquad-output/merged"
# ----------------------

# quantization config (matches your config.json)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

def generate_answer(question):
    prompt = (
        "<|im_start|>user\n"
        f"{question}\n"
        "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.2,
        do_sample=False
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    # Return only the assistant part
    return decoded.split("assistant")[-1].strip()

# Gradio UI
ui = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(label="Ask a medical question"),
    outputs=gr.Textbox(label="Answer"),
    title="Medical Q&A using Qwen2 (Fine-tuned)",
)

ui.launch()


/opt/anaconda3/lib/python3.13/site-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


ImportError: The installed version of bitsandbytes (<0.43.1) requires CUDA, but CUDA is not available. You may need to install PyTorch with CUDA support or upgrade bitsandbytes to >=0.43.1.